In [80]:
from pulp import *

In [81]:
def ReadFile(f_name):
    info = []
    Machines = []
    Locations = []
    Requests = []
    Technicians = []
    i = -1
    j = -1
    with open(f_name) as infile:
        for line in infile:
            i += 1
            if len(line.split()) == 0:
                next
            else:
                if line.split()[0] == 'MACHINES':
                    machines = int(line.split()[2])
                if line.split()[0] == 'LOCATIONS':
                    locations = int(line.split()[2])
                if line.split()[0] == 'REQUESTS':
                    requests = int(line.split()[2])
                if line.split()[0] == 'TECHNICIANS':
                    technicians = int(line.split()[2])
    with open(f_name) as infile:
        for line in infile:
            j += 1
            if len(line.split()) == 0:
                next
            else:
                if j > 2 and j < 13:
                    info.append(line.split()[0])
                    info.append(int(line.split()[2]))
                if j > 14 and j <= 14 + machines:
                    temp = line.split()
                    for e in range(0,len(temp)):
                        temp[e] = int(temp[e])
                    Machines.append(temp)
                if j > 16 + machines and j <= 16 + machines + locations:
                    temp = line.split()
                    for e in range(0,len(temp)):
                        temp[e] = int(temp[e])
                    Locations.append(temp)
                if j > 18 + machines + locations and j <= 18 + machines + locations + requests:
                    temp = line.split()
                    for e in range(0,len(temp)):
                        temp[e] = int(temp[e])
                    Requests.append(temp)
                if j > 20 + machines + locations + requests and j <= 20 + machines + locations + requests + technicians:
                    temp = line.split()
                    for e in range(0,len(temp)):
                        temp[e] = int(temp[e])
                    Technicians.append(temp)
    return info, Machines, Locations, Requests, Technicians



def Formula(Locations,i,j):
    ret = ((Locations[i][1]-Locations[j][1])**2 + (Locations[i][2]-Locations[j][2])**2)**0.5
    return round(ret,0)


def Distances(Locations):
    mydict = {}
    for i in range(len(Locations)):
        for j in range(len(Locations)):
            if i != j:
                mydict[(i,j)] = Formula(Locations,i,j)
            else:
                mydict[(i,j)] = 0
    return mydict

def DistancesIncludingTechnicianHome(Locations, LocationOfTechnicians):
    mydict = {}
    for i in range(len(Locations)):
        for j in range(len(Locations)):
            for s in range(len(LocationOfTechnicians)):
                if i==0 and j > 0:
                    mydict[i,j,s]=Formula(Locations,LocationOfTechnicians[s]-1,j)
                elif i>0 and j==0:
                    mydict[i,j,s]=Formula(Locations,i,LocationOfTechnicians[s]-1)
                else:
                    mydict[(i,j,s)]= Formula(Locations,i,j)
    return mydict

    

def DistancesForTechnicianAndRequest(departReq, deliveryReq, technician, techLocationList, distancesMatrix,LocationOfRequests): 
    # we need such a function to cover the technicians' different locations
    distance=0
    techLocation = techLocationList[technician-1]
    if departReq == 0 and deliveryReq !=0:
        distance=distancesMatrix[(techLocation-1,LocationOfRequests[deliveryReq-1]-1)]
    if departReq != 0 and deliveryReq ==0:
        distance=distancesMatrix[(LocationOfRequests[departReq-1]-1,techLocation-1)]
    if departReq >0 and deliveryReq>0:
        distance=distancesMatrix[(LocationOfRequests[departReq-1]-1,LocationOfRequests[deliveryReq-1]-1)]
    return distance

def getColumn(aList, whichColumn):
    outputList=list()
    for i in range(len(aList)):
        outputList.append(aList[i][whichColumn])
    return outputList

def compatibilityFunc(SetANDParameters, machineTypeListOfRequests, noOfTechnicians):
    compatibilityOutput=list()
    for i in range(noOfTechnicians):
        rowList=list()
        for j in range(len(machineTypeListOfRequests)):
            if getColumn(SetANDParameters[4],3+machineTypeListOfRequests[j])[i] == 1 :
                rowList.append(1)
            else:
                rowList.append(0)
        compatibilityOutput.append(rowList)
    return compatibilityOutput
            


In [82]:
setsAndParameters= ReadFile("test.txt")

numberOfRequests=len(setsAndParameters[3])
numberofTechnicians=len(setsAndParameters[4])
numberOfMachines=len(setsAndParameters[1])

DAYS=setsAndParameters[0][1]
TRUCK_CAPACITY=setsAndParameters[0][3]
TRUCK_MAX_DISTANCE=setsAndParameters[0][5]
TRUCK_DISTANCE_COST=setsAndParameters[0][7]
TRUCK_DAY_COST=setsAndParameters[0][9]
TRUCK_COST=setsAndParameters[0][11]
TECHNICIAN_DISTANCE_COST=setsAndParameters[0][13]
TECHNICIAN_DAY_COST=setsAndParameters[0][15]
TECHNICIAN_COST=setsAndParameters[0][17]

              ## related to the machines ##
sizeOfMachines=getColumn(setsAndParameters[1],1)
penaltyCostMac=getColumn(setsAndParameters[1],2)

              ## related to the requests ##
locationOfRequestsTrucks=getColumn(setsAndParameters[3],1)
locationOfRequestsTechnicians=locationOfRequestsTrucks[:]
locationOfRequestsTrucks.insert(0,1)# the reason to insert the element "1" to the requests is that the request "0" is the depot which is not included in the requests matrix in the data set.
lOR=locationOfRequestsTrucks[:]
earliestTime=getColumn(setsAndParameters[3],2)  # earliest time of the request j; length is numberOfRequests
latestTime  =getColumn(setsAndParameters[3],3)  # latest time of the request j; length is numberOfRequests
machineType =getColumn(setsAndParameters[3],4)
numOfMachine=getColumn(setsAndParameters[3],5)  # capacity needed for the request j 
#capacityOfRequest=getColumn(setsAndParameters[3],5)
distances  = Distances(setsAndParameters[2])    # distance between the request locations, including the depot
              
              ## related to the technician ##
maxDistanceTechnician=getColumn(setsAndParameters[4],2)
maxRequestTechnician =getColumn(setsAndParameters[4],3)
locationID_OfTechnician=getColumn(setsAndParameters[4],1)

              ##   technician & request   ##
a=compatibilityFunc(setsAndParameters,machineType,numberofTechnicians)
distancesIncludingHomes=DistancesIncludingTechnicianHome(setsAndParameters[2],locationID_OfTechnician)



## Parameters

In [83]:
R = range(1,numberOfRequests+1)  # set of requests without the depot
R_0 = range(numberOfRequests+1) # set of requests with the depot 
S = range(1,numberofTechnicians+1) # set of technicians
K = range(1,numberOfRequests+1) # set of vehicles !!the number of vehicles is not specified in the project instructions.
Days = range(1,DAYS+1) # set of days

### MODEL

In [84]:
model = LpProblem("VRP_ExactApproachh", LpMinimize)

## Decision Variables

In [85]:
x = LpVariable.dicts("X",
                     [(i,j,k,t) for i in R_0
                                for j in R_0
                                for k in K
                                for t in Days],
                     0,1,LpBinary)  ## request --> request 
z = LpVariable.dicts("Z",             ######## !!!!!!!!!!!!!!!!!!!!!!!!! i and i from ????
                     [(i,j,s,t) for i in R_0
                                for j in R_0
                                for s in S
                                for t in Days],
                     0,1,LpBinary)  ## request --> request ?
D = LpVariable.dicts("Delivery",
                    [(i,t) for i in R
                           for t in Days],
                    0,1,LpBinary)   ## delivery related

I = LpVariable.dicts("Installation",
                    [(i,t) for i in R
                           for t in Days],
                    0,1,LpBinary)   ## installation related

q = LpVariable.dicts("CapacityRelated",
                    [(i,k) for i in R_0
                           for k in K],
                    0,TRUCK_CAPACITY)   ## capacity of vehicle k after delivery request j

u = LpVariable.dicts("SubTourEliminatTechnician",
                    [(i,s)  for i in R_0
                            for s in S],
                     0,100)            ## a decision variable for subtour elimination of the technicians
n = LpVariable.dicts("n",
                    [(k,t) for k in K
                           for t in Days],
                    0,1,LpBinary)        ## for counting the trucks for each day
P = LpVariable.dicts("P",
                    [(s,t) for s in S
                           for t in Days],
                    0,1,LpBinary)        ## for counting the technicians for each day
Pi = LpVariable.dicts("Pi", S, 0, 1, LpBinary) ## for counting the technicians
ne = LpVariable.dicts("ne", K, 0, 1, LpBinary) ## for counting the technicians



penaltyCost= LpVariable.dicts("Penalty", R,0, 10000) # total penalty cost values of the requests


## Objective Function

In [86]:
model += lpSum(x[(i,j,k,t)]*distances[(locationOfRequestsTrucks[i]-1,locationOfRequestsTrucks[j]-1)]*TRUCK_DISTANCE_COST for i in R_0 for j in R_0 for k in K for t in Days if i !=j)\
+lpSum(z[(i,j,s,t)]*distancesIncludingHomes[(lOR[i]-1,lOR[j]-1,s-1)]*TECHNICIAN_DISTANCE_COST for i in R_0 for j in R_0 for s in S for t in Days if i !=j)+\
lpSum(n[(k,t)] for t in Days for k in K)*TRUCK_DAY_COST+ lpSum(ne[k] for k in K)*TRUCK_COST+\
lpSum(P[(s,t)] for t in Days for s in S)*TECHNICIAN_DAY_COST+ lpSum(Pi[s] for s in S)*TECHNICIAN_COST+ \
lpSum(penaltyCost[i]*numOfMachine[i-1]*penaltyCostMac[machineType[i-1]-1] for i in R)    
        #!! z part is problematic

## Constraints

In [87]:
 ########### Ensuring the deliveries and installations are satisfied and add earliest and latest times constraints
for j in R:
    model += lpSum(x[(i,j,k,t)] for i in R_0 for k in K for t in range(earliestTime[j-1],latestTime[j-1]+1) if i != j)==1,""
    
for j in R:
    model += lpSum(z[(i,j,s,t)] for i in R_0 for s in S for t in Days if i != j)==1,""
    
    ################### balance equations ######################
for j in R_0:
    for t in Days:
        for k in K:
            model += lpSum(x[(i,j,k,t)] for i in R_0 if i != j) == lpSum(x[(j,i,k,t)] for i in R_0 if i != j),""       
for j in R_0:
    for t in Days:
        for s in S:
            model += lpSum(z[(i,j,s,t)] for i in R_0 if i != j) == lpSum(z[(j,i,s,t)] for i in R_0 if i != j),"" 
            
        ########### technician type constraints ############

for j in R:
    for s in S:
        model += lpSum(z[(i,j,s,t)] for i in R_0 for t in Days if i != j) <= a[s-1][j-1],"" #new
                    
         ####### Maximum Truck Distance   
for t in Days:
    for k in K:
        model += lpSum(x[(i,j,k,t)]*distances[(locationOfRequestsTrucks[i]-1,locationOfRequestsTrucks[j]-1)] \
                       for i in R_0 for j in R_0 if i != j) <= TRUCK_MAX_DISTANCE,""
        
        ######## Maximum Travelling Limitations for Technicians ##########  i,j -->
for t in Days:
    for s in S:
        model += lpSum(z[(i,j,s,t)]*distancesIncludingHomes[(lOR[i]-1,lOR[j]-1,s-1)] for i in R_0 for j in R_0 if i !=j)\
        <= maxDistanceTechnician[s-1],""
        
       ######## Maximum Number of Requests for Technicians ##########  i,j -->
for t in Days:
    for s in S:
        model += lpSum(z[(i,j,s,t)] for i in R_0 for j in R if i != j) <= maxRequestTechnician[s-1],""
        


        ##### earliest and latest time constraints #######
#for i in R:
#    model += lpSum(D[(i,t)]*t for t in Days) <=latestTime[i-1],""
    
#for i in R:
#    model += lpSum(D[(i,t)]*t for t in Days) >=earliestTime[i-1],""
    
       ############# time related and linking x s and D s ############ 
for j in R:
    for t in Days:
        model += D[(j,t)] == lpSum(x[(i,j,k,t)] for k in K for i in R_0 if i!=j),""

for i in R:
    model += lpSum(D[(i,t)] for t in Days)==1,""

     ############# time related and linking z s and I s ############ 
for j in R:
    for t in Days:
        model += I[(j,t)] == lpSum(z[(i,j,s,t)] for s in S for i in R_0 if i!=j),""

for i in R:
    model += lpSum(I[(i,t)] for t in Days)==1,""
    
    # penalty cost calculation
for i in R:
    model += lpSum(I[(i,t)]*t for t in Days)-lpSum(D[(i,t)]*t for t in Days)-1 == penaltyCost[i],""
    
    ## number of trucks and technician used constraints ####
    
for i in R_0:
    for j in R_0:
        for k in K:
            for t in Days:
                if i != j:
                    model += n[(k,t)] >= x[(i,j,k,t)],""
for i in R_0:
    for j in R_0:
        for s in S:
            for t in Days:
                if i != j:
                    model += P[(s,t)] >= z[(i,j,s,t)],""
for k in K:
    for t in Days:
        model += ne[k]>= n[(k,t)],""
        
for s in S:
    for t in Days:
        model += Pi[s]>= P[(s,t)],""
        
        ################################# consecutive days and off constraints for the technicians #####################3

for t in range(1,DAYS-5+1):
    for s in S:
        model += lpSum(P[(s,tt)] for tt in range(t,t+5))<=5 - P[(s,t+5)],""
        


In [88]:
##########  Subtour Elimination For the Trucks ##########

for i in R_0:
    for j in R:
        for t in Days:
            for k in K:
                if i !=j:
                    model += q[(j,k)]-TRUCK_CAPACITY <= q[(i,k)]-(x[(i,j,k,t)]*(TRUCK_CAPACITY+(numOfMachine[j-1]*sizeOfMachines[machineType[j-1]-1]))),""
for k in K:
    model += q[(0,k)]==TRUCK_CAPACITY,""
    
##########  Subtour Elimination For the Technicians ##########
for i in R_0:
    for j in R:
        for t in Days:
            for s in S:
                if i !=j:
                    model += u[(j,s)]-maxRequestTechnician[s-1] <= u[(i,s)]-(z[(i,j,s,t)]*(1+maxRequestTechnician[s-1])),""
for s in S:
    model += u[(0,s)]==maxRequestTechnician[s-1],""

for s in S:
    for t in Days:
        model += P[(s,t)]==lpSum(z[(0,j,s,t)] for j in R),""
        
for s in S:
    for t in Days:
        model += P[(s,t)]==lpSum(z[(j,0,s,t)] for j in R),""



In [ ]:
model.solve()
pulp.LpStatus[model.status]


In [11]:
for v in model.variables():
    if v.varValue >= 1:
        print(v.name, "=", v.varValue)
print()
print("objective value: ", model.objective.value())

CapacityRelated_(0,_1) = 6.0
CapacityRelated_(0,_2) = 6.0
CapacityRelated_(0,_3) = 6.0
CapacityRelated_(0,_4) = 6.0
CapacityRelated_(0,_5) = 6.0
CapacityRelated_(0,_6) = 6.0
CapacityRelated_(0,_7) = 6.0
CapacityRelated_(1,_1) = 4.0
CapacityRelated_(5,_1) = 3.0
CapacityRelated_(6,_1) = 4.0
Delivery_(1,_2) = 1.0
Delivery_(2,_4) = 1.0
Delivery_(3,_2) = 1.0
Delivery_(4,_5) = 1.0
Delivery_(5,_4) = 1.0
Delivery_(6,_5) = 1.0
Delivery_(7,_5) = 1.0
Installation_(1,_3) = 1.0
Installation_(2,_5) = 1.0
Installation_(3,_3) = 1.0
Installation_(4,_6) = 1.0
Installation_(5,_5) = 1.0
Installation_(6,_6) = 1.0
Installation_(7,_6) = 1.0
P_(1,_3) = 1.0
P_(1,_5) = 1.0
P_(2,_3) = 1.0
P_(2,_6) = 1.0
P_(4,_6) = 1.0
Pi_1 = 1.0
Pi_2 = 1.0
Pi_4 = 1.0
SubTourEliminatTechnician_(0,_1) = 2.0
SubTourEliminatTechnician_(0,_2) = 1.0
SubTourEliminatTechnician_(0,_3) = 5.0
SubTourEliminatTechnician_(0,_4) = 2.0
SubTourEliminatTechnician_(0,_5) = 1.0
SubTourEliminatTechnician_(2,_1) = 1.0
SubTourEliminatTechnician_(3,_1)

In [57]:
## CALCULATION OF TOTAL TRUCK ROUTE
xList=list()
for v in model.variables():
    if v.name[0:1]=="X" and v.varValue==1:
        xList.append(v.name)
firstList=list()
secondList=list()
for e in xList:
    firstList.append(int(e[3:4]))
    secondList.append(int(e[6:7]))
totalRoute=0
for i in range(0,len(firstList)):
    totalRoute += distances[(firstList[i],secondList[i])]
    print(distances[(firstList[i],secondList[i])])
print("TOTAL TRUCK ROUTE:", totalRoute)
  

41.0
32.0
61.0
17.0
13.0
60.0
49.0
32.0
40.0
31.0
45.0


In [49]:
## CALCULATION OF TOTAL TECHNICIAN ROUTE
zList=list()
for v in model.variables():
    if v.name[0:1]=="Z" and v.varValue==1:
        zList.append(v.name)
firsttList=list()
seconddList=list()
thirdList=list()
for e in zList:
    firsttList.append(int(e[3:4]))
    seconddList.append(int(e[6:7]))
    thirdList.append(int(e[9:10]))
totalRoutee=0
for i in range(0,len(firsttList)):
    totalRoutee += distancesIncludingHomes[(firstList[i],secondList[i],thirdList[i])]
print("Total Technician Route: ", totalRoutee)

IndexError: list index out of range